In [1]:
!pip install datasets

In [2]:
#Loading the huggingface dataset
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("SKNahin/bengali-transliteration-data")

README.md:   0%|          | 0.00/300 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/333k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5006 [00:00<?, ? examples/s]

In [3]:
!pip install indic-nlp-library

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.4/587.4 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 71.5 MB/s eta 0:00:00
  Attempting uninstall: docutils
    Found existing installation: docutils 0.18.1
    Uninstalling docutils-0.18.1:
      Successfully uninstalled docutils-0.18.1
  Attempting uninstall: sphinx
    Found existing installation: Sphinx 5.0.2
    Uninstalling Sphinx-5.0.2:
      Successfully uninstalled Sphinx-5.0.2


In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer
from indicnlp.tokenize import sentence_tokenize

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [5]:
# Perform the split
train_test_split = dataset['train'].train_test_split(test_size=0.2, seed=42)

# Separate into train and test sets
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

In [6]:
print(dataset['train'][:5])

{'bn': ['স্ক্রোল করে ২০/৩০ সেকেন্ড এর ভিডিও পান নাই???', 'ও গুলা টরেন্ট সাইট এ পাবেন', 'ভক্কর চক্কর পোস্ট একটা করলেই এপ্রুভড.… নিশ্চই  ঘাবলা আছে', 'আমি টেস্ট করেই কোড দিছি…', 'এতো কষ্টের কি আছে সাকিবওয়াপ.টক,সাকিবওয়াপ.মল&এআইওভিডিওড্ল.মল থেকে সহজেই ডাউনলোড করা যায়'], 'rm': ['scroll kore 20/30 second er video pann nai???', 'o gula Torrent site e paben', 'vokkor chokkor post akta korlei approved…. nishchoi ghabla ache', 'ami test koreii code disi…', 'eto koster ki ache shakibwap.tk,shakibwap.ml&aiovideodl.ml theke shohojei downlod kora jay']}


In [7]:
banglish_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
def bengali_tokenizer(text):
    # Simple whitespace tokenization
    return text.split()

In [9]:
def tokenize_and_filter(example, max_len=100, min_len=3):
    # Tokenize Romanized Bengali (rm)
    example['rm_tokens'] = bengali_tokenizer(example['rm'])
    # Tokenize Bengali (bn)
    example['bn_tokens'] = bengali_tokenizer(example['bn'])

    example['labels'] = example['bn_tokens'] 
    
    return example

In [10]:
# Tokenize the dataset
tokenized_dataset = dataset['train'].map(
    tokenize_and_filter,
    remove_columns=['bn', 'rm'],  # Remove original columns
    batched=False
)

min_len = 3
max_len = 100
# Filter rows based on token lengths
filtered_dataset = tokenized_dataset.filter(
    lambda x: min_len <= len(x['rm_tokens']) <= max_len and min_len <= len(x['bn_tokens']) <= max_len
)


# Verify the result
print(filtered_dataset[:5])

Map:   0%|          | 0/5006 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5006 [00:00<?, ? examples/s]

{'rm_tokens': [['scroll', 'kore', '20/30', 'second', 'er', 'video', 'pann', 'nai???'], ['o', 'gula', 'Torrent', 'site', 'e', 'paben'], ['vokkor', 'chokkor', 'post', 'akta', 'korlei', 'approved….', 'nishchoi', 'ghabla', 'ache'], ['ami', 'test', 'koreii', 'code', 'disi…'], ['eto', 'koster', 'ki', 'ache', 'shakibwap.tk,shakibwap.ml&aiovideodl.ml', 'theke', 'shohojei', 'downlod', 'kora', 'jay']], 'bn_tokens': [['স্ক্রোল', 'করে', '২০/৩০', 'সেকেন্ড', 'এর', 'ভিডিও', 'পান', 'নাই???'], ['ও', 'গুলা', 'টরেন্ট', 'সাইট', 'এ', 'পাবেন'], ['ভক্কর', 'চক্কর', 'পোস্ট', 'একটা', 'করলেই', 'এপ্রুভড.…', 'নিশ্চই', 'ঘাবলা', 'আছে'], ['আমি', 'টেস্ট', 'করেই', 'কোড', 'দিছি…'], ['এতো', 'কষ্টের', 'কি', 'আছে', 'সাকিবওয়াপ.টক,সাকিবওয়াপ.মল&এআইওভিডিওড্ল.মল', 'থেকে', 'সহজেই', 'ডাউনলোড', 'করা', 'যায়']], 'labels': [['স্ক্রোল', 'করে', '২০/৩০', 'সেকেন্ড', 'এর', 'ভিডিও', 'পান', 'নাই???'], ['ও', 'গুলা', 'টরেন্ট', 'সাইট', 'এ', 'পাবেন'], ['ভক্কর', 'চক্কর', 'পোস্ট', 'একটা', 'করলেই', 'এপ্রুভড.…', 'নিশ্চই', 'ঘাবলা', 'আছে'], ['আমি', '

In [11]:
split_datasets = filtered_dataset.train_test_split(test_size=0.2)
train_dataset = split_datasets["train"]
val_dataset = split_datasets["test"]

In [12]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In [13]:
# Split the dataset into train and validation sets (80-20 split)
split_datasets = filtered_dataset.train_test_split(test_size=0.2)
train_dataset = split_datasets["train"]
val_dataset = split_datasets["test"]

# Check the dataset after preprocessing
print(train_dataset[0]) 

{'rm_tokens': ['ei', 'range', 'e', 'realme', 'narzo', '30', 'best'], 'bn_tokens': ['এই', 'রেঞ্জ', 'এ', 'রিয়েলমি', 'নাৰ্জো', '৩০', 'বেস্ট'], 'labels': ['এই', 'রেঞ্জ', 'এ', 'রিয়েলমি', 'নাৰ্জো', '৩০', 'বেস্ট']}


In [14]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./mbart50-banglish-bangla",  # Save the fine-tuned model here
    evaluation_strategy="epoch",  # Evaluate after each epoch
    learning_rate=5e-5,
    per_device_train_batch_size=8,  # Adjust based on your GPU memory
    per_device_eval_batch_size=8,
    num_train_epochs=3,  # Number of epochs to train
    weight_decay=0.01,
    save_total_limit=2,
    predict_with_generate=True,  # Generate text (important for sequence-to-sequence tasks)
    logging_dir="./logs",
    logging_steps=500,  # How often to log
    fp16=True,  # Use mixed precision for faster training
)

# Set up the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Start training
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


NameError: name 'model' is not defined